In [79]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from random import randrange

F = Zmod(65537)

# Returns B, D such that B * D = M with D having Gaussian coefficients of the specified standard deviation.
def trapdoor(M:Matrix, sigma:float):
    D = gaussian(M.nrows(), M.ncols(), sigma)
    return M * D^-1, D

def gaussian(rows, cols:int, sigma:float):
    X = DiscreteGaussianDistributionIntegerSampler(sigma)
    return matrix(F, rows, cols, lambda i, j: X())


l = 3
d = 3
u =  Matrix([[1, 1, 1]])
witness = [F(randrange(0, 2)) for i in range(l)]

M = [[0, 0] for i in range(l)]
for i in range(l):
    M[i][0] = matrix(F, d, d, lambda i,j: F(randrange(0, 2)))
    M[i][1] = matrix(F, d, d, lambda i,j: F(randrange(0, 2)))


def Gen(u, M, x):
    
    n = len(x)
    
    def gen(M, A):
        S = matrix(F, d, d, lambda i,j: F.random_element(2))
        MS = M.tensor_product(S)
        A_prev, D = trapdoor(MS*A,3.2)#+gaussian(MS.nrows(), MS.ncols(), 3.2)
        return S, A_prev, D

    S = [0 for i in range(n)]
    A = [0 for i in range(n)]
    D = [[0, 0] for i in range(n-1)]
    
    print(n)

    #A[i-1] = ((M[i] x S[i])*A[i] + E)*D[i]^-1
    
    for i in range(n-1, 0, -1):
        
        if i == n-1:
            A[i] = matrix(F, d*d, d*d, lambda i, j: F.random_element())

        if x[i] == 0:
            S[i], A[i-1], D[i-1][0] = gen(M[i][0], A[i])
            _, _, D[i-1][1] = gen(M[i][1], A[i])
        else:
            _, _, D[i-1][0] = gen(M[i][0], A[i])
            S[i], A[i-1], D[i-1][1] = gen(M[i][1], A[i])
        
        if i == n-1:
            S_witness = S[i]
        else:
            S_witness *= S[i]
        
        print(A[i-1] * D[i-1][0] - M[i][0].tensor_product(S[i])*A[i])
        print(A[i-1] * D[i-1][1] - M[i][1].tensor_product(S[i])*A[i])
        
    # C0 = (uM0 x S0) * A0 + E
    C = [0, 0]
    S0 = matrix(F, d, d, lambda i,j: F.random_element(2))
    S1 = matrix(F, d, d, lambda i,j: F.random_element(2))
    if x[0] == 0:
        S[0] = S0
    else:
        S[0] = S1
        
    C[0] = (u * M[0][0]).tensor_product(S0) * A[0] #+= gaussian(d*d, d*d, 3.2)
    C[1] = (u * M[0][1]).tensor_product(S1) * A[0] #+= gaussian(d*d, d*d, 3.2)

    S_witness *= S[0]
    
    CHECK = u
    for i in range(n):
        CHECK *= M[i][x[i]]
    CHECK = CHECK.tensor_product(S_witness) * A[n-1]

    return C, D, CHECK

def Eval(C, D, CHECK, witness):
    
    OUT = C[witness[0]]
    for i in range(len(witness)-1):
        OUT *= D[i][witness[i+1]]
    return OUT - CHECK

C, D, CHECK = Gen(u, M, witness)

print()
print(Eval(C, D, CHECK, witness))

#Example for l = 3
# C0 * D1 * D2
# = ((uM0 x S0) * A0 + E) * D1 * D2
# = ((uM0 x S0) * ((M1 x S1)*A1 + E)*D1^-1 + E) * D1 * D2
# = ((uM0 x S0) * ((M1 x S1)*A1 + E) + E*D1) * D2
# = ((uM0 x S0) * ((M1 x S1)*((M2 x S2)*A2 + E)*D2^-1 + E) + E*D1) * D2
# = ((uM0 x S0) * ((M1 x S1)*((M2 x S2)*A2 + E) + E*D2) + E*D1*D2)
# = ((uM0 x S0) * (M1 x S1) * (M2 x S2) * A2 + [E + E*D2 + E*D1*D2]
# = ((uM0 * M1 * M2) x (S0 * S1 * S2))*A2 + E




3
[ 7493 48056 15572 65286 28730 10664 20460 62119 43803]
[28340 64038   944 46681  7474 54966 37249 29819 26369]
[53671 30255 13457 64076 55626 53384 52579   169  9528]
[53219 47037 27788 43538 55621 34371 49648 25548 27838]
[ 5223  7477  6833  7063 34383 58570 33045 28982 62228]
[31911  6551 47538 18430 24515 17671  2134 49794 36939]
[39050 25209  4377 20178 43555 39845 51388 63330 61487]
[35333 33716 40194 28823 38676 62483 22329 38740 24665]
[12505 36544 13434  6174 18620 46647 51995 19536 25481]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[46292 10876  2401 17735 30030 22207 53253 47301  2132]
[12313  2363 60199 21365 18182 28819 63592 18755  5303]
[41815  3123 48567 27912 53044 49005 43171 60690  7658]
[14999 37436 62536 21901  7599  1706 30081 26531 33781]
[ 7618 30046  5183 59856  7481 62556 43505 37609 53736]
[46093 12939 58109 18171 44132 373